In [48]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

# 1 读入数据
读入文本，并小写化, one-hot

In [49]:
raw_text = open('./test.txt').read()
raw_text =  raw_text.lower()

In [50]:
chars = sorted(set(raw_text))
char_to_int =  dict((c, i) for i ,c in enumerate(chars))
int_to_chars = dict((i,c) for i, c in enumerate(chars))

In [51]:
int_to_chars

{0: '\n',
 1: ' ',
 2: '!',
 3: '#',
 4: "'",
 5: '(',
 6: ')',
 7: '*',
 8: ',',
 9: '-',
 10: '.',
 11: '0',
 12: '1',
 13: '2',
 14: '3',
 15: '4',
 16: '5',
 17: '6',
 18: '7',
 19: '8',
 20: '9',
 21: ':',
 22: ';',
 23: '?',
 24: '[',
 25: ']',
 26: 'a',
 27: 'b',
 28: 'c',
 29: 'd',
 30: 'e',
 31: 'f',
 32: 'g',
 33: 'h',
 34: 'i',
 35: 'j',
 36: 'k',
 37: 'l',
 38: 'm',
 39: 'n',
 40: 'o',
 41: 'p',
 42: 'q',
 43: 'r',
 44: 's',
 45: 't',
 46: 'u',
 47: 'v',
 48: 'w',
 49: 'x',
 50: 'y',
 51: 'z',
 52: '“',
 53: '”',
 54: '\ufeff'}

In [52]:
len(chars), len(raw_text)

(55, 19504)

在这片文章中，一共有60个字符

我们的目标是，给了前置的字母后，下一个字母是什么？


# 2 构造训练集、测试集

需要把raw_text 编程可以用来训练的x，y
x是前置字母，y是后一个字母


In [53]:
seq_length = 100
x =[]
y= []
for i in range(0,  len(raw_text) - seq_length):
    given =  raw_text[i:i+seq_length]
    predict = raw_text[i+seq_length]
    x.append([char_to_int[char] for char in given])
    y.append(char_to_int[predict])


### 我们可以看看做好的数据集

In [54]:
print(x[:1])
print(y[:1])

[[54, 45, 33, 30, 1, 41, 43, 40, 35, 30, 28, 45, 1, 32, 46, 45, 30, 39, 27, 30, 43, 32, 1, 30, 27, 40, 40, 36, 1, 40, 31, 1, 45, 33, 30, 1, 28, 43, 34, 44, 34, 44, 8, 1, 28, 40, 38, 41, 37, 30, 45, 30, 8, 1, 27, 50, 1, 48, 34, 39, 44, 45, 40, 39, 1, 28, 33, 46, 43, 28, 33, 34, 37, 37, 0, 24, 26, 46, 45, 33, 40, 43, 1, 34, 44, 1, 45, 33, 30, 1, 26, 38, 30, 43, 34, 28, 26, 39, 1, 48]]
[34]


### 接下来的两步处理
 
1 已经有了input的数字表达，我们要把它变成LSTM 需要的数组格式:[样本数，时间步伐，特征]      
2 对于output，用one-hot做output的预测可以给我们更好的效果，相对于直接预测一个准确的y数值的话

In [55]:
n_patterns =  len(x)
n_vocab =  len(chars)
# 把x变成LSTM需要的格式
x=  np.reshape(x, [n_patterns,seq_length, 1]) # 1 表示每个特征用1维 表示（0-60）

# normal 处理到0-1之间
x =  x / float(n_vocab)
# y转化成one-hot形式
y = np_utils.to_categorical(y)
print(x[11])
print(y[11])

[[0.81818182]
 [0.01818182]
 [0.58181818]
 [0.83636364]
 [0.81818182]
 [0.54545455]
 [0.70909091]
 [0.49090909]
 [0.54545455]
 [0.78181818]
 [0.58181818]
 [0.01818182]
 [0.54545455]
 [0.49090909]
 [0.72727273]
 [0.72727273]
 [0.65454545]
 [0.01818182]
 [0.72727273]
 [0.56363636]
 [0.01818182]
 [0.81818182]
 [0.6       ]
 [0.54545455]
 [0.01818182]
 [0.50909091]
 [0.78181818]
 [0.61818182]
 [0.8       ]
 [0.61818182]
 [0.8       ]
 [0.14545455]
 [0.01818182]
 [0.50909091]
 [0.72727273]
 [0.69090909]
 [0.74545455]
 [0.67272727]
 [0.54545455]
 [0.81818182]
 [0.54545455]
 [0.14545455]
 [0.01818182]
 [0.49090909]
 [0.90909091]
 [0.01818182]
 [0.87272727]
 [0.61818182]
 [0.70909091]
 [0.8       ]
 [0.81818182]
 [0.72727273]
 [0.70909091]
 [0.01818182]
 [0.50909091]
 [0.6       ]
 [0.83636364]
 [0.78181818]
 [0.50909091]
 [0.6       ]
 [0.61818182]
 [0.67272727]
 [0.67272727]
 [0.        ]
 [0.43636364]
 [0.47272727]
 [0.83636364]
 [0.81818182]
 [0.6       ]
 [0.72727273]
 [0.78181818]
 [0.01

In [56]:
x.shape, y.shape

((19404, 100, 1), (19404, 54))

# 3 模型构造

In [57]:
model = Sequential()
model.add(LSTM(128, input_shape=(x.shape[1], x.shape[2])))
model.add(Dropout(0.2))

# Dense 普通神经网络
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

## 跑模型

In [58]:
model.fit(x,y,epochs=3, batch_size=32)

Epoch 1/3
19404/19404 [==============================] - 266s 14ms/step - loss: 3.0969
Epoch 2/3
19404/19404 [==============================] - 280s 14ms/step - loss: 3.0528
Epoch 3/3
19404/19404 [==============================] - 262s 14ms/step - loss: 3.0141


In [69]:
def predict_next(input_array):
    x= np.reshape(input_array, (1, seq_length, 1))
    x=  x / float(n_vocab)
    y= model.predict(x)
    print(y)
    
    return y
 
def string_to_index(raw_input):
    res = []
    for c in raw_input[(len(raw_input) - seq_length):]:
        res.append(char_to_int[c])
    return res
def y_to_char(y):
    largest_index = y.argmax()
    c=  int_to_chars[largest_index]
    return c

def generate_article(init, rounds = 500):
    in_string =  init.lower()
    for i in range(rounds):
#         print(string_to_index(in_string))
        
        n = y_to_char(predict_next(string_to_index(in_string)))
#         print(n)
        in_string += n
        return in_string
init  = "Professor Michael S. Hart is the originator of the Project, maybe it is wrong, who knows it ? but today, i will go to school for more classess."
article =  generate_article(init)
print(article)


[[2.80270949e-02 2.32596666e-01 1.37920585e-03 9.38836602e-05
  4.31722170e-03 1.98163485e-04 1.80859119e-04 5.64400281e-04
  9.11289547e-03 2.14155088e-03 9.01869126e-03 2.58445507e-04
  1.64730416e-04 3.88212327e-04 1.68561310e-04 1.02739454e-04
  2.01195144e-04 5.40220644e-04 9.38182347e-05 3.06231435e-04
  1.03787119e-04 6.10166171e-04 3.59595899e-04 1.67671987e-03
  8.18534463e-05 1.43622950e-04 4.70640585e-02 1.79775562e-02
  3.69328111e-02 3.14375907e-02 6.13404065e-02 1.22343320e-02
  1.96383428e-02 2.94149257e-02 3.11857387e-02 1.03970105e-03
  7.23259943e-03 3.38479318e-02 1.97832994e-02 2.85461433e-02
  2.81914752e-02 2.07826961e-02 7.50475738e-04 3.57069522e-02
  7.05087855e-02 9.62974057e-02 1.24528455e-02 7.16165639e-03
  3.02299857e-02 2.47567263e-03 1.21897468e-02 4.68607584e-04
  6.39636861e-03 5.88131929e-03]]
professor michael s. hart is the originator of the project, maybe it is wrong, who knows it ? but today, i will go to school for more classess. 
